In [6]:
pip install ordered-set

Note: you may need to restart the kernel to use updated packages.


In [7]:
pip install matplotlib

Note: you may need to restart the kernel to use updated packages.


In [8]:
pip install torch

Note: you may need to restart the kernel to use updated packages.


In [9]:
pip install transformers

Note: you may need to restart the kernel to use updated packages.


In [10]:
pip install datasets

Note: you may need to restart the kernel to use updated packages.


In [11]:
pip install shap

In [12]:
pip install scikit-learn

Note: you may need to restart the kernel to use updated packages.


In [13]:
import re

In [14]:
import pandas as pd
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import torch
import transformers
import datasets
import shap

In [15]:
from string import punctuation

In [4]:
import sys
sys.path.append('SHAPHaabsa_plus_plus')

In [16]:
from IPython.display import clear_output

In [17]:
from ordered_set import OrderedSet
from collections import Counter

# Prepare The Data

In [18]:
with open('SHAPHaabsa_plus_plus/data_shap.txt', 'r') as file:
    data = file.readlines()

In [19]:
tokenizer = transformers.AutoTokenizer.from_pretrained("nateraw/bert-base-uncased-emotion", use_fast=True)

In [20]:
new_data = list()

for i in range(0, len(data), 3):

    text = data[i].strip()
    text = tokenizer.decode(tokenizer.convert_tokens_to_ids(re.sub('_\d+', '', text).split(' ')))

    target_words = data[i+1].strip()
    target_words = tokenizer.decode(tokenizer.convert_tokens_to_ids(re.sub('_\d+', '', target_words).split(' ')))
#     for target_word in re.split(r'(?<!\() (?!\(|\))', target_words):

    original_text = text.replace(' [UNK]', '').replace('[UNK]', '').strip()

#     target_tokens = list()
#     for t in tokenizer.tokenize(target_word):
#         if t in punctuation:
#             t = f'\{t}'
#         target_tokens.append(re.findall(f'{t}_\d+', data[i+1].strip())[0])
#     target_tokens = ' '.join(target_tokens)
    target_tokens = data[i+1].strip()
        

    new_data.append(original_text)
    new_data.append(data[i].strip())
    new_data.append(target_tokens)
    new_data.append(data[i+2].strip())

In [21]:
new_data = [_+ '\n' for _ in new_data]

In [22]:
new_data[-1] = new_data[-1].strip()

In [23]:
with open('SHAPHaabsa_plus_plus/768remainingtestdata2016_processed.txt', 'w') as file:
    file.writelines(new_data)

# Predictable Model

In [24]:
# def load_inputs_twitter(input_file, word_id_file, sentence_len, type_='', is_r=True, target_len=10, encoding='utf8'):
#     word_to_id = word_id_file #word_id_file is the list with words (including token number) and its corresponding row number in the embedding file
#     print('load word-to-id done!')

#     x, y, sen_len = [], [], []
#     x_r, sen_len_r = [], []
#     target_words = []
#     tar_len = []
#     all_target, all_sent, all_y = [], [], []
#     # read in txt file
#     lines = open(input_file).readlines() #open data_predicted file and makes a list of strings per line. Ends with \n after each line to indicate new line
#     for i in range(0, len(lines), 4): #len(lines) = amount of lines in the text file. Currently 248*3 lines
#         # targets
#         clear_output(wait=True)
#         print(i)
#         words = lines[i + 2].lower().split() #creates a list of all the (sub)words in the aspect
#         target = words

#         target_word = []
#         for w in words: #amount of words in the aspect
#             if w in word_to_id:
#                 target_word.append(word_to_id[w]) #appends the line/row numbers of the (sub)words in the aspect
#         l = min(len(target_word), target_len) #l = the amount of (sub)words in the aspect, up to a maximum of 10
#         tar_len.append(l) #tar_len is a list of amount of sub(words) in each aspect, up to a maximum of 10
#         target_words.append(target_word[:l] + [0] * (target_len - l)) #target_words is a list of the line/row numbers of each sub(word) in the aspect; the list consists of 10 numbers, with the remaining numbers becoming 0 if there are no more words left in the aspect

#         # sentiment
#         y.append(lines[i + 3].strip().split()[0]) #list of sentiments

#         # left and right context
#         words = lines[i+1].lower().split() #a list of the (sub)words, split between each (sub)word
#         sent = words
#         words_l, words_r = [], []
#         flag = True
#         for word in words:
#             if word == '$t$':
#                 flag = False
#                 continue
#             if flag:
#                 if word in word_to_id:
#                     words_l.append(word_to_id[word]) #appends all the line/row numbers of the (sub)words that come before $t$, aka the left part
#             else:
#                 if word in word_to_id:
#                     words_r.append(word_to_id[word]) #appends all the line/row numbers of the (sub)words that come after $t$, aka the right part
#         # words_l.extend(target_word)
#         words_l = words_l[:sentence_len] #sentence_len = 80, the first 80 (sub)word tokens in the left part
#         words_r = words_r[:sentence_len] #sentence_len = 80, the first 80 (sub)word tokens in the right part
#         sen_len.append(len(words_l)) #the length of the left sentences appended in a list
#         x.append(words_l + [0] * (sentence_len - len(words_l))) #x is a list of the line/row numbers of the (sub)words in the left part, with maximum 80; less than 80 means the rest is filled with zero's
#         # tmp = target_word + words_r
#         tmp = words_r #the first 80 (sub)word tokens in the right part
#         tmp.reverse() #Reverses the order of the list of tokens in the right part
#         sen_len_r.append(len(tmp)) #the length of the right sentences appended in a list
#         x_r.append(tmp + [0] * (sentence_len - len(tmp))) #x_r is a list of the line/row numbers of the (sub)words in the right part, with maximum 80; less than 80 means the rest is filled with zero's
#         all_sent.append(sent) #a list of a list of all sub(words) of the sentence
#         all_target.append(target) #a list of a list of all sub(words) of the aspect

#     all_y = y; #list of all sentiments ['1', '1', ... , ]
#     y = change_y_to_onehot(y) #array of sentiment labels, as a list of three [[1, 0, 0], [1, 0, 0], [1, 0, 0], ... ] but listed vertically


#     if type_ == 'TD':
#         return np.asarray(x), np.asarray(sen_len), np.asarray(x_r), \
#                np.asarray(sen_len_r), np.asarray(y)
#     elif type_ == 'TC': #TC currently
#         return np.asarray(x), np.asarray(sen_len), np.asarray(x_r), np.asarray(sen_len_r), \
#                np.asarray(y), np.asarray(target_words), np.asarray(tar_len), np.asarray(all_sent), np.asarray(all_target), np.asarray(all_y)
#     elif type_ == 'IAN':
#         return np.asarray(x), np.asarray(sen_len), np.asarray(target_words), \
#                np.asarray(tar_len), np.asarray(y)
#     else:
#         return np.asarray(x), np.asarray(sen_len), np.asarray(y)

In [25]:
# def getStatsFromFile(index):
#     polarity_vector = []
#     with open(path, "r") as fd:
#         lines = fd.read().splitlines()
#         size = len(lines) / 4
#         for i in range(0, len(lines), 4):
#             # polarity
#             polarity_vector.append(lines[i + 3].strip().split()[0])
#     return size, polarity_vector

In [26]:
# input_file = 'SHAPHaabsa_plus_plus/test.txt'

# x_left, x_left_len, x_right, x_right_len, y_true, target_word, \
#         target_words_len, _, _, _ = load_inputs_twitter(input_file, word_id_mapping, 80, 'TC', '1', 19)

In [27]:
# size, polarity = getStatsFromFile(input_file)
# size, len(polarity)

In [28]:
def load_w2v(w2v_file, embedding_dim=768, is_skip=False):
    fp = open(w2v_file)
    if is_skip:
        fp.readline()
    w2v = []
    word_dict = dict()
    # [0,0,...,0] represent absent words
    w2v.append([0.] * embedding_dim)
    cnt = 0
    for line in fp:
        cnt += 1
        line = line.split()
        # line = line.split()
        if len(line) != embedding_dim + 1:
            print('a bad word embedding: {}'.format(line[0]))
            continue
        w2v.append([float(v) for v in line[1:]])
        word_dict[line[0]] = cnt
    w2v = np.asarray(w2v, dtype=np.float32)
    w2v = np.row_stack((w2v, np.sum(w2v, axis=0) / cnt))
    print(np.shape(w2v))
    word_dict['$t$'] = (cnt + 1)
    # w2v -= np.mean(w2v, axis=0)
    # w2v /= np.std(w2v, axis=0)
    print(word_dict['$t$'], len(w2v))
    return word_dict, w2v

In [29]:
embedding_file = "SHAPHaabsa_plus_plus/768embedding2016.txt"

In [30]:
word_id_mapping, w2v = load_w2v(embedding_file, 768)

(51065, 768)
51064 51065


In [31]:
tokens_data = open('SHAPHaabsa_plus_plus/768remainingtestdata2016_processed.txt').readlines()

In [32]:
lines = tokens_data

In [33]:
input_texts = ['the entire was extremely accomodating and tended to my every need']
# valid_tokens = tokenizer.tokenize(input_texts[0])

In [34]:
import random

In [35]:
input_text_stripped = input_texts[0].replace(' ... ', '.*').replace(' ...', '.*').replace('... ', '.*').replace('...', '.*')

print('[^A-Za-z0-9]*'+input_text_stripped+'[^A-Za-z0-9]*')

re.findall('[^A-Za-z0-9]*'+input_text_stripped+'[^A-Za-z0-9]*', lines[8])#[0]

[^A-Za-z0-9]*the entire was extremely accomodating and tended to my every need[^A-Za-z0-9]*


[]

In [36]:
# all_matches = list()
# for input_text in input_texts:
#     sentence_match = list()
#     input_text_stripped = input_text.replace(' ... ', '.*').replace(' ...', '.*').replace('... ', '.*').replace('...', '.*')
#     for i in range(0, len(lines), 4):
#         sentence_match.extend(re.findall('[^A-Za-z0-9]*'+input_text_stripped+'[^A-Za-z0-9]*', lines[i]))
#     all_matches.append(sentence_match)

# true_match = all_matches[0]
# for j in range(1, len(all_matches)):
#     true_match = np.intersect1d(true_match, all_matches[j])
# true_match = [_ for _ in true_match if len(_) != 0]
# true_match = random.choice(true_match)#[0]

In [37]:
# i = np.argwhere(np.array(lines) == true_match)[0, 0]

In [38]:
# words = lines[i+1].lower().split()

In [39]:
## OLD
# words_only = [_.split('_')[0] for _ in words]
# indexes = []
# for ji in range(len(words_only) - len(valid_tokens) + 1):
#     if valid_tokens == words_only[j:j+len(valid_tokens)]:
#         indexes = list(range(j, j+len(valid_tokens)))
#         break

In [40]:
# words_only = [_.split('_')[0] for _ in words]
# indexes = [np.argwhere(_ == np.array(words_only)) for _ in valid_tokens]

# valid_indexes = list()
# for ind in range(len(indexes)):
#     if len(indexes[ind]) == 0:
#         continue
#     if (ind == len(indexes) -1) and (indexes[ind].shape[0] > 1):
#         valid_indexes.append(indexes[ind][np.argwhere(indexes[ind][:, 0] > valid_indexes[-1])[0, 0]][0])
#     elif (indexes[ind].shape[0] > 1):
#         valid_indexes.append(indexes[ind][np.argwhere(indexes[ind][:, 0] < indexes[ind+1][0, 0])[0, 0]][0])
#     else:
#         valid_indexes.append(indexes[ind][0, 0])

# indexes = valid_indexes
# indexes

In [41]:
def load_input_data(input_texts, sentence_len, type_='', is_r=True, target_len=10, encoding='utf8'):
    word_to_id = word_id_mapping.copy()

    x, y, sen_len = [], [], []
    x_r, sen_len_r = [], []
    target_words = []
    tar_len = []
    all_target, all_sent, all_y = [], [], []
    # read in txt file
    lines = tokens_data #open data_predicted file and makes a list of strings per line. Ends with \n after each line to indicate new line
    
    all_matches = list()
    for input_text in input_texts:
        sentence_match = list()
        input_text_stripped = input_text.replace(' ... ', '.*').replace(' ...', '.*').replace('... ', '.*').replace('...', '.*')
        for i in range(0, len(lines), 4):
            sentence_match.extend(re.findall('[^A-Za-z0-9]*'+input_text_stripped+'[^A-Za-z0-9]*', lines[i]))
        all_matches.append(sentence_match)

    true_match = all_matches[0]
    for j in range(1, len(all_matches)):
        true_match = np.intersect1d(true_match, all_matches[j])
    true_match = [_ for _ in true_match if len(_) != 0]
    true_match = random.choice(true_match)
    if true_match == '':
        return None
    i = np.argwhere(np.array(lines) == true_match)[0, 0]
    
    
    for input_text in input_texts:
        input_text_stripped = input_text.replace('...', '')

        valid_tokens = tokenizer.tokenize(input_text_stripped)
    #     print(valid_tokens)

        words = lines[i + 2].lower().split() #creates a list of all the (sub)words in the aspect
        target = words

        target_word = []
        for w in words: #amount of words in the aspect
#             if w.split('_')[0] not in valid_tokens:
#                 continue
            if w in word_to_id:
                target_word.append(word_to_id[w]) #appends the line/row numbers of the (sub)words in the aspect
        l = min(len(target_word), target_len) #l = the amount of (sub)words in the aspect, up to a maximum of 10
        tar_len.append(l) #tar_len is a list of amount of sub(words) in each aspect, up to a maximum of 10
        target_words.append(target_word[:l] + [0] * (target_len - l)) #target_words is a list of the line/row numbers of each sub(word) in the aspect; the list consists of 10 numbers, with the remaining numbers becoming 0 if there are no more words left in the aspect

        # sentiment
        y.append(lines[i + 3].strip().split()[0]) #list of sentiments

        # left and right context
        words = lines[i+1].lower().split() #a list of the (sub)words, split between each (sub)word
        sent = words
        words_l, words_r = [], []
        flag = True
        
        words_only = [_.split('_')[0] for _ in words]
        indexes = [np.argwhere(_ == np.array(words_only)) for _ in valid_tokens]
        valid_indexes = list()
        for ind in range(len(indexes)):
            if len(indexes[ind]) == 0:
                continue
            if (ind == len(indexes) -1) and (indexes[ind].shape[0] > 1):
                valid_indexes.append(indexes[ind][np.argwhere(indexes[ind][:, 0] > valid_indexes[-1])[0, 0]][0])
            elif (indexes[ind].shape[0] > 1):
                valid_indexes.append(indexes[ind][np.argwhere(indexes[ind][:, 0] < indexes[ind+1][0, 0])[0, 0]][0])
            else:
                valid_indexes.append(indexes[ind][0, 0])
        indexes = valid_indexes
#         print('index', indexes)
        
        for j, word in enumerate(words):
            if word == '$t$':
                flag = False
                continue
            if j not in indexes:
                continue
            if flag:
                if word in word_to_id:
                    words_l.append(word_to_id[word]) #appends all the line/row numbers of the (sub)words that come before $t$, aka the left part
            else:
                if word in word_to_id:
                    words_r.append(word_to_id[word]) #appends all the line/row numbers of the (sub)words that come after $t$, aka the right part
        # words_l.extend(target_word)
        words_l = words_l[:sentence_len] #sentence_len = 80, the first 80 (sub)word tokens in the left part
        words_r = words_r[:sentence_len] #sentence_len = 80, the first 80 (sub)word tokens in the right part
        sen_len.append(len(words_l)) #the length of the left sentences appended in a list
        x.append(words_l + [0] * (sentence_len - len(words_l))) #x is a list of the line/row numbers of the (sub)words in the left part, with maximum 80; less than 80 means the rest is filled with zero's
        # tmp = target_word + words_r
        tmp = words_r #the first 80 (sub)word tokens in the right part
        tmp.reverse() #Reverses the order of the list of tokens in the right part
        sen_len_r.append(len(tmp)) #the length of the right sentences appended in a list
        x_r.append(tmp + [0] * (sentence_len - len(tmp))) #x_r is a list of the line/row numbers of the (sub)words in the right part, with maximum 80; less than 80 means the rest is filled with zero's
        all_sent.append(sent) #a list of a list of all sub(words) of the sentence
        all_target.append(target) #a list of a list of all sub(words) of the aspect

    all_y = y; #list of all sentiments ['1', '1', ... , ]
    y = change_y_to_onehot(y) #array of sentiment labels, as a list of three [[1, 0, 0], [1, 0, 0], [1, 0, 0], ... ] but listed vertically

    size = 1
    polarity_vector = [lines[i + 3].strip().split()[0]]
    
    return np.asarray(x), np.asarray(sen_len), np.asarray(x_r), np.asarray(sen_len_r), \
               np.asarray(y), np.asarray(target_words), np.asarray(tar_len), size, polarity_vector

In [42]:
def change_y_to_onehot(y):
#     print(Counter(y)) #Counter({'1': 144, '-1': 82, '0': 22})
    class_set = OrderedSet(['1','-1','0']) #Positive, negative, neutral
    n_class = len(class_set) #3
    y_onehot_mapping = dict(zip(class_set, range(n_class))) #{'1': 0, '-1': 1, '0': 2}
#     print(y_onehot_mapping)
    onehot = []
    for label in y:
        tmp = [0] * n_class #temporary list of zero's [0, 0, 0]
        tmp[y_onehot_mapping[label]] = 1 #temporary list of sentiment labels e.g. [1, 0, 0] for positive
        onehot.append(tmp) #append the sentiment to onehot; results in a list of a list of sentiment labels e.g. [[1, 0, 0], [1, 0, 0], [1, 0, 0], ... ]
    return np.asarray(onehot, dtype=np.int32) 

In [43]:
model_path = 'SHAPHaabsa_plus_plus/Model_graph/-188'

In [44]:
def orderProb(probabilities):
    """
    convert probs from the classifier into the right format: [neg,neu,pos]
    """
    nr, nc = probabilities.shape
    correctProb = np.zeros((nr, nc))

    correctProb[:, 1] = probabilities[:, 2]
    correctProb[:, 2] = probabilities[:, 0]
    correctProb[:, 0] = probabilities[:, 1]

    return correctProb

In [45]:
def get_allProb(x_left, x_left_len, x_right, x_right_len, y_true, target_word, target_words_len, batch_size,
                num_samples):
    
    tf.reset_default_graph()
    # import the loaded graph
    imported_graph = tf.train.import_meta_graph(model_path + '.meta')
    config = tf.ConfigProto(allow_soft_placement=True)
    config.gpu_options.allow_growth = True

    sess = tf.Session(config=config)
    sess.run(tf.global_variables_initializer())
    # restore the saved variables
    imported_graph.restore(sess, model_path)
    graph = tf.get_default_graph()
    
    x = graph.get_tensor_by_name('inputs/Placeholder:0')
    y = graph.get_tensor_by_name('inputs/Placeholder_1:0')
    sen_len = graph.get_tensor_by_name('inputs/Placeholder_2:0')
    x_bw = graph.get_tensor_by_name('inputs/Placeholder_3:0')
    sen_len_bw = graph.get_tensor_by_name('inputs/Placeholder_4:0')
    target_words = graph.get_tensor_by_name('inputs/Placeholder_5:0')
    tar_len = graph.get_tensor_by_name('inputs/Placeholder_6:0')
    keep_prob1 = graph.get_tensor_by_name('Placeholder:0')
    keep_prob2 = graph.get_tensor_by_name('Placeholder_1:0')
    prob = graph.get_tensor_by_name('softmax/Softmax:0')
    
    probabilities = np.zeros((num_samples, 3))
    predictions = np.zeros(num_samples)

#     for i in range(int(num_samples / batch_size)):
#         batch_start = i * batch_size
#         batch_end = (i + 1) * batch_size

    feed_dict = {
        x: x_left, # x_left is also x_fw or x
        x_bw: x_right,
        y: y_true,
        sen_len: x_left_len, # sen_len is also sen_len_fw
        sen_len_bw: x_right_len,
        target_words: target_word,
        tar_len: target_words_len,
        keep_prob1: 1,
        keep_prob2: 1,
    }
    
    if x_left.shape[0] == 1:
        for k,v in feed_dict.items():
            if type(v) is np.ndarray:
                feed_dict[k] = np.concatenate([v, v], 0)
        

    ##getting prediction of instance

    prob = sess.run(prob, feed_dict=feed_dict) # prob matrix for all instances for the sentim (n_inst x 3)
    prob = orderProb(prob) # convert probs from the classifier into the right format: [neg,neu,pos]

    pred = np.argmax(prob, axis=1) - 1 # highest prob class chosen and coverted from 0, 1, 2 to corr sent

    probabilities = prob
    predictions = pred
    
    print(x_left.shape)
    if x_left.shape[0] == 1:
        return probabilities[[0]]
    return probabilities

In [46]:
import tensorflow as tf

Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.


In [47]:
def prediction(text):
#     print(text)
    x_left, x_left_len, x_right, x_right_len, y_true, target_word, \
        target_words_len, size, polarity = load_input_data(text, 80, 'TC', '1', 19)
    probabilities = get_allProb(x_left, x_left_len, x_right, x_right_len, y_true, target_word, target_words_len, int(size), int(size))
    return probabilities

# SHAP Integration

In [48]:
dataset = tokens_data[::4]
dataset = [_.strip() for _ in dataset]

In [49]:
labels = ['positive', 'negative', 'neutral']

In [50]:
def custom_tokenizer(s, return_offsets_mapping=True):

    print('tokenizer:', s)
    """ Custom tokenizers conform to a subset of the transformers API.
    """
    pos = 0
    offset_ranges = []
    input_ids = []
    for m in re.finditer(r"\W", s):
        start, end = m.span(0)
        offset_ranges.append((pos, start))
        input_ids.append(s[pos:start])
        pos = end
    if pos != len(s):
        offset_ranges.append((pos, len(s)))
        input_ids.append(s[pos:])
    out = {}
    out["input_ids"] = input_ids
    if return_offsets_mapping:
        out["offset_mapping"] = offset_ranges
    return out

masker = shap.maskers.Text(custom_tokenizer)
explainer = shap.Explainer(prediction, masker, output_names=labels)

tokenizer: 
tokenizer: ...


In [51]:
shap_values = explainer(dataset[2:3])

tokenizer: for the amount of we got the prices should have been lower.
INFO:tensorflow:Restoring parameters from SHAPHaabsa_plus_plus/Model_graph/-188
(1, 80)
INFO:tensorflow:Restoring parameters from SHAPHaabsa_plus_plus/Model_graph/-188
(1, 80)
INFO:tensorflow:Restoring parameters from SHAPHaabsa_plus_plus/Model_graph/-188
(2, 80)


  0%|                                                                                                                                                                                                                                                                                          | 0/132 [00:00<?, ?it/s]

INFO:tensorflow:Restoring parameters from SHAPHaabsa_plus_plus/Model_graph/-188


  8%|████████████████████▋                                                                                                                                                                                                                                                            | 10/132 [00:07<01:29,  1.36it/s]

(6, 80)
INFO:tensorflow:Restoring parameters from SHAPHaabsa_plus_plus/Model_graph/-188


...

 26%|██████████████████████████████████████████████████████████████████████▎                                                                                                                                                                                                          | 34/132 [00:36<01:53,  1.16s/it]

(6, 80)
INFO:tensorflow:Restoring parameters from SHAPHaabsa_plus_plus/Model_graph/-188


 30%|██████████████████████████████████████████████████████████████████████████████████▋                                                                                                                                                                                              | 40/132 [00:44<01:48,  1.18s/it]

(6, 80)


IndexError: list index out of range

In [37]:
# prediction(dataset[:5])

In [38]:
# shap_values

In [503]:
shap.plots.text(shap_values)

# SHAP Testing

In [172]:
# load the emotion dataset
dataset_test  = datasets.load_dataset("SetFit/emotion", split = "train")
data_test = pd.DataFrame({'text':dataset_test['text'],'emotion':dataset_test['label']})

Using custom data configuration SetFit--emotion-e88ff06c56f54cb4
Reusing dataset json (/home/rev9/.cache/huggingface/datasets/SetFit___json/SetFit--emotion-e88ff06c56f54cb4/0.0.0/a3e658c4731e59120d44081ac10bf85dc7e1388126b92338344ce9661907f253)


In [404]:
# tokenizer.decode(tokenizer.convert_tokens_to_ids(tokenizer.tokenize('green tea creme brulee')))

In [173]:
tokenizer_test = transformers.AutoTokenizer.from_pretrained("nateraw/bert-base-uncased-emotion", use_fast=True)
model_test = transformers.AutoModelForSequenceClassification.from_pretrained("nateraw/bert-base-uncased-emotion")#.cuda()
labels_test = sorted(model_test.config.label2id, key=model_test.config.label2id.get)

In [174]:
# this defines an explicit python function that takes a list of strings and outputs scores for each class
def f(x):
    print('function:', x)
#     print('func-token:', [tokenizer.encode(v) for v in x])
    tv = torch.tensor([tokenizer_test.encode(v, padding='max_length', max_length=128, truncation=True) for v in x])#.cuda()
    attention_mask = (tv!=0).type(torch.int64)#.cuda()
    outputs = model_test(tv,attention_mask=attention_mask)[0].detach().cpu().numpy()
    scores = (np.exp(outputs).T / np.exp(outputs).sum(-1)).T
    val = sp.special.logit(scores)
    return val

In [175]:
f(['text']).shape

function: ['text']


(1, 6)

In [177]:
labels_test

['sadness', 'joy', 'love', 'anger', 'fear', 'surprise']

In [178]:
method = "custom tokenizer"

# build an explainer by passing a transformers tokenizer
if method == "transformers tokenizer":
    explainer = shap.Explainer(f, tokenizer, output_names=labels)

# build an explainer by explicitly creating a masker
elif method == "default masker":
    masker = shap.maskers.Text(r"\W") # this will create a basic whitespace tokenizer
    explainer = shap.Explainer(f, masker, output_names=labels)

# build a fully custom tokenizer
elif method == "custom tokenizer":
    import re

    def custom_tokenizer(s, return_offsets_mapping=True):
        
        print('tokenizer:', s)
        """ Custom tokenizers conform to a subset of the transformers API.
        """
        pos = 0
        offset_ranges = []
        input_ids = []
        for m in re.finditer(r"\W", s):
            start, end = m.span(0)
            offset_ranges.append((pos, start))
            input_ids.append(s[pos:start])
            pos = end
        if pos != len(s):
            offset_ranges.append((pos, len(s)))
            input_ids.append(s[pos:])
        out = {}
        out["input_ids"] = input_ids
        if return_offsets_mapping:
            out["offset_mapping"] = offset_ranges
        return out

    masker = shap.maskers.Text(custom_tokenizer)
    explainer = shap.Explainer(f, masker, output_names=labels)

tokenizer: 
tokenizer: ...


In [40]:
labels

['positive', 'negative', 'neutral']

In [ ]:
'green tea creme brulee is a'

In [ ]:
'green tea creme brulee must'

In [98]:
shap_values = explainer(['is a must!'])

tokenizer: is a must!
['...']
INFO:tensorflow:Restoring parameters from SHAPHaabsa_plus_plus/Model_graph/-9212
(1, 80)
['is a must']
INFO:tensorflow:Restoring parameters from SHAPHaabsa_plus_plus/Model_graph/-9212
(1, 80)
['is a ...' '...must']
INFO:tensorflow:Restoring parameters from SHAPHaabsa_plus_plus/Model_graph/-9212
(2, 80)
['is ...' '...a ...' 'is ...must' '...a must']
INFO:tensorflow:Restoring parameters from SHAPHaabsa_plus_plus/Model_graph/-9212
(4, 80)


  0%|          | 0/6 [00:00<?, ?it/s]

Partition explainer: 2it [00:16, 16.56s/it]               

tokenizer: is a must!


In [180]:
shap_values

.values =
array([[[-0.19676185, -0.27401131, -0.24243718,  0.3072746 ,
         -0.04136848, -0.01146805],
        [-0.0922811 , -0.95139172, -0.42734808,  0.58435331,
          0.03147972, -0.36040938],
        [ 1.21712346,  0.52153998, -0.37354755, -3.30336308,
         -0.81317073, -0.40101993],
        [ 6.53970294, -6.29361225, -2.65164495, -3.61388195,
         -3.75852233, -2.77599418]]])

.base_values =
array([[-1.84233999, -0.2942284 , -3.64444399, -0.67128611, -2.76130891,
        -4.31995296]])

.data =
(array(['i ', 'didnt ', 'feel ', 'humiliated'], dtype=object),)

In [18]:
shap.plots.text(shap_values)

# Create Embeddings

In [33]:
from transformers import AutoTokenizer, AutoModel
import torch

In [40]:
#Mean Pooling - Take attention mask into account for correct averaging
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)


# Sentences we want sentence embeddings for
sentences = ['This is an example sentence', 'Each sentence is converted']

# Load model from HuggingFace Hub
tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/bert-base-nli-mean-tokens')
model = AutoModel.from_pretrained('sentence-transformers/bert-base-nli-mean-tokens')

# Tokenize sentences
encoded_input = tokenizer(sentences, padding=True, truncation=True, return_tensors='pt')

# Compute token embeddings
with torch.no_grad():
    model_output = model(**encoded_input)

# Perform pooling. In this case, max pooling.
sentence_embeddings = mean_pooling(model_output, encoded_input['attention_mask'])

print("Sentence embeddings:")

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Sentence embeddings:


In [38]:
sentence_embeddings[0].shape

torch.Size([768])